# Step 2: IRFs

In this notebook, we will run magic-cta-pipe scripts on a small data sample. Due to time constraints, we are unable to run the pipeline on all the files needed to produce the main plots (e.g. SED, light curves), so we will provide you a complete dataset to get 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the paths where you want to save logging information.

In [1]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

### Instrument Response Function  

Here we will create point-like IRFs (using ring-wobble MC test sample, applying optimal gammaness and $\theta$ cuts).
The IRFs (e.g.: effective area and energy migration matrix,...) will be produced in the fitz.gz format which is then used in the next steps (DL3 and DL4).
If gamma diffuse test sample is provided it is possible to calculate the offset-dependent IRFs (Full-enclosure). To get the point-like IRFs the  **fov_offset_bins** should be set to a single bin.

We can run the script directly in the terminal with:

$ python lst1_magic_create_irf.py -g yourfile -o Path1 -c configfile

with the following options:

-g: input DL2 gamma file (test sample)

-o: output directory (to save IRFs)

-c: configuration file:

In the configuration file, we have to set the parameters as shown below:

1) IRF **quality_cuts**: "disp_diff_mean < 0.22", which represent the standard quality cut on DISPs.
2) **event_type**: "software", where the options are "hardware" which stands for a hardware trigger between MAGIC and LST-1; "software", which stands for any 2- or 3-telescope combinations except MAGIC-I + MAGIC-II; "software_only_3tel", which stands only for the combo of the type MI + MII+ LST-1; and "magic_only".
3) **cut_type**: "dynamic". The options are "global", for a cut that is not energy dependent ; "dynamic", for a cut evaluated in every energy bin according to the given gamma and theta efficiencies.

In [2]:
%%capture
os.system("sed -n 214,257p /fefs/aswg/workspace/analysis-school-2024/MCP/irf/conf/config.yaml")

create_irf:
    quality_cuts: "disp_diff_mean < 0.22"
    event_type: "software"  # select "software", "software_only_3tel", "magic_only" or "hardware"
    weight_type_dl2: "intensity"  # select "simple", "variance" or "intensity"
    obs_time_irf: "50 h"  # used when creating a background HDU

    energy_bins:  # log space
        start: "0.01 TeV"
        stop: "1000 TeV"
        n_edges: 26

    migration_bins:  # log space
        start: 0.2
        stop: 5
        n_edges: 31

    fov_offset_bins:  # linear space, used for diffuse MCs
        start: "0 deg"
        stop: "1 deg"
        n_edges: 2

    source_offset_bins:  # linear space, used when creating PSF HDU
        start: "0 deg"
        stop: "1 deg"
        n_edges: 101

    bkg_fov_offset_bins:  # linear space, used when creating background HDU
        start: "0 deg"
        stop: "10 deg"
        n_edges: 21

    gammaness:
        cut_type: "dynamic"  # select "global" or "dynamic"
        global_cut_value: 0.8  # use

Let's start by setting up the configuration and data paths:

In [3]:

dir_irf=('....../IRF')
#dir_irf='/fefs/aswg/workspace/elisa.visentin/sw_school_2024/IRF'
os.makedirs(dir_irf)

f=open(f'{dir_irf}/IRF.log','w')

input_gamma=('/fefs/aswg/workspace/analysis-school-2024/MCP/dl2/output/dl2_gamma_zd_37.814deg_az_270.0deg_LST1_MAGIC1_MAGIC2_run10102_to_20002.h5')

config=('/fefs/aswg/workspace/analysis-school-2024/MCP/irf/conf/config.yaml')

Lines to get files from the data directory

In [4]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()

Here we use python subprocess.run to run the script and also get a log file

In [5]:
for input_file in input_file_gamma: 
    x=subprocess.run(['lst1_magic_create_irf', f'-g{input_file}', f'-o{dir_irf}',\
        f'-c{config}'], stdout=f, stderr=f) 

We can check the log file by typing:

In [6]:
more $dir_irf/IRF.log